# GPT3 prompt engineering
I've been having a lot of trouble getting consistently formatted predictions out of GPT3, so here I'm going to play with changing the prompts interactively.
<br><br>
A lit of the issues I've been having with various prompts:
* Triples without quotes enclosing the elements
* Some documents not getting predictions
    * I'm not sure if this has always been happening -- I thought I had previously been getting predictions for all of them, but I was only printing the string with the pred itself and not the entire response json, so it's possible I thought I was but just didn't notice that there weren't actually predictions for all of them
* All documents not getting predictions
    * This happens if I provide just. a string for the abstract, instead of a string enclosed as a list, but also happens other times for unknown reasons
    * Another reason this seems to happen is if I indicate that I want it to put the sentence index outside the triple tuple, instead of as a fourth element in the tuple
* Not able to get triple elements in quotes while also getting the sentence index
    * This happens when I ask for the sentence index as the fourth element of the tuple

**Description of sentence index problem and motivation:** I previously was using a function I wrote to identify the sentences from whence each triple came. However, the function was initially based on the assumption that when linked together, the three parts of the triple would form one contiguous span within the sentence. This is not the case, and while it's more or less straightforward to allow the three components to disjoint, doing so means that in many cases, it's not possible to uniquely identify the original sentence. For example, if we were trying to identify the origin sentence for the triple `("Jasmonic acid", "activates", "Protein 1")` in the following text:

```
Jasmonic acid very often activates Protein 1. Jasmonic acid also activates Protein 1 in cases where Protein 2 is deactivated.
```
we would be unable to tell if the triple came from the first or second sentence. There are two potential solutions for this for a function exterior to GPT3, which is (1) only use the first instance, or (2) put it as a triple in both sentences. However, both of these options have the potential to skew the performance evaluation towards better or worse recall than the actual recall. So, if I'm able to get GPT3 to just tell me where it got the triple from, I can accurately reflect how many triples are being correctly identified by the model.

In [1]:
from os import listdir
from os.path import isfile
import sys
sys.path.append('../benchmark_scripts/')
from gpt3_openai import gpt3_predict
import jsonlines

First, let's read in the documents we want to make predictions on:

In [2]:
abst_dir = '../data/gold_std/test_txt_docs/'
abstracts = {}
for f in listdir(abst_dir):
    if isfile(f'{abst_dir}/{f}'):
        with open(f'{abst_dir}/{f}') as myf:
            abstracts[f] = [l.strip() for l in myf.readlines()]

Now, let's look at the output from the prompt I was using before trying to add the sentence number to the equation:

In [3]:
prompt = ('Extract the biological relationships from '
     'the following text as (Subject, Predicate, Object) triples: '
     '{abstract_txt}.\nExample relationship triple:\n("Protein 1", '
     '"regulates", "Protein 2")')

In [4]:
#gpt3_predict(abstracts, prompt)

It looks like this prompt is getting no predictions. However, I was previously getting properly formatted predictions with this prompt. This implies that the model's response to the same prompt is not deterministic, which is quite frustrating. Additionally, earlier today I got an unknown server error, which is apparently [currently happening to a lot of people](https://community.openai.com/t/ratelimiterror-the-server-had-an-error-with-no-reason-given/50146/14) with the `text-davinci-003` model. What happens if I try the `text-curie-001` model like some folks in the thread have said works?
<br><br>
UPDATE: When I tried running the above cell again, it now hangs after the second abstract. Not totally sure what's going on here, but going to kill it to move on to the next cell.

In [5]:
gpt3_predict(abstracts, prompt, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:07,  1.37it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973247,
  "id": "cmpl-6i7UVDKmqbRAAnDslsnQt52Xt82Vw",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 47,
    "total_tokens": 47
  }
}
------------------------------------------



 18%|█▊        | 2/11 [00:01<00:05,  1.52it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973247,
  "id": "cmpl-6i7UVb8pDZJg39qD7k2nqj0YRIh1p",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 47,
    "total_tokens": 47
  }
}
------------------------------------------



 27%|██▋       | 3/11 [00:01<00:04,  1.74it/s]

...

 91%|█████████ | 10/11 [00:03<00:00,  3.98it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973250,
  "id": "cmpl-6i7UYu63mBtMMppIGhkyQ2YFlJznd",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 47,
    "total_tokens": 47
  }
}
------------------------------------------



100%|██████████| 11/11 [00:03<00:00,  3.00it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973250,
  "id": "cmpl-6i7UYN1QGuq0SacKXBv7EggfWdhZi",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 47,
    "total_tokens": 47
  }
}
------------------------------------------



{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UVDKmqbRAAnDslsnQt52Xt82Vw at 0x2b989f5d7fc0> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": ""
     }
   ],
   "created": 1675973247,
   "id": "cmpl-6i7UVDKmqbRAAnDslsnQt52Xt82Vw",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "prompt_tokens": 47,
     "total_tokens": 47
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UVb8pDZJg39qD7k2nqj0YRIh1p at 0x2b989f5fc830> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": ""
     }
   ],
   "created": 1675973247,
   "id": "cmpl-6i7UVb8pDZJg39qD7k2nqj0YRIh1p",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "prompt_tokens": 47,
     "total_tokens": 47
   }
 },
 'PMID15869647_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UWf

These are also all empty. Let's quickly return to basics and try with the sample prompt:

In [6]:
dummy_prompt = 'Say this is a test'
gpt3_predict(abstracts, dummy_prompt, model='text-curie-001', max_tokens=1500)

 18%|█▊        | 2/11 [00:00<00:01,  4.80it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1675973250,
  "id": "cmpl-6i7UYcK8KjeznVEXJImtDT7fJdmyo",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}
------------------------------------------


This is a test.

Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1675973250,
  "id": "cmpl-6i7UYaIC1SVDmT2LlUKzfPConnqZe",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}
-----

 27%|██▋       | 3/11 [00:00<00:01,  4.95it/s]


Response and prediction for PMID15869647_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1675973251,
  "id": "cmpl-6i7UZUZ2gRymgByMT1oh8Jq2Zg627",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}
------------------------------------------


This is a test.


 36%|███▋      | 4/11 [00:00<00:01,  4.71it/s]

...

 91%|█████████ | 10/11 [00:02<00:00,  4.10it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1675973253,
  "id": "cmpl-6i7Ub0wMAqR6cAGuIC7NeHHaZNdFB",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}
------------------------------------------


This is a test.


100%|██████████| 11/11 [00:02<00:00,  4.04it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1675973253,
  "id": "cmpl-6i7UbdVfSX2MyeZHOhxmhRH7YY4Ie",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}
------------------------------------------


This is a test.


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UYcK8KjeznVEXJImtDT7fJdmyo at 0x2b989f667fc0> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nThis is a test."
     }
   ],
   "created": 1675973250,
   "id": "cmpl-6i7UYcK8KjeznVEXJImtDT7fJdmyo",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 7,
     "prompt_tokens": 5,
     "total_tokens": 12
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UYaIC1SVDmT2LlUKzfPConnqZe at 0x2b989f897150> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nThis is a test."
     }
   ],
   "created": 1675973250,
   "id": "cmpl-6i7UYaIC1SVDmT2LlUKzfPConnqZe",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 7,
     "prompt_tokens": 5,
     "total_t

That worked, which is reassuring. Now, let's try slowly building up the sample prompt to do what I want, and see if I can identify the failure point. For now, I'm going to continue using `text-curie-001` because `text-davinci-003` keeps hanging.


In [7]:
prompt_step1 = 'Extract the semantic triples from the following text: "This is a test"'
gpt3_predict(abstracts, prompt_step1, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:04,  2.42it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
    }
  ],
  "created": 1675973253,
  "id": "cmpl-6i7Ub8QqVgM5dv8L7nIsMbldi7VPY",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}
------------------------------------------


This is a test
This is a sentence
This is a text


 18%|█▊        | 2/11 [00:00<00:03,  2.49it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
    }
  ],
  "created": 1675973253,
  "id": "cmpl-6i7UbrZn4hgrDNukvQuRtUH7ZHbSV",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}
------------------------------------------


This is a test
This is a sentence
This is a text


 27%|██▋       | 3/11 [00:01<00:03,  2.49it/s]

...

 91%|█████████ | 10/11 [00:03<00:00,  2.82it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
    }
  ],
  "created": 1675973256,
  "id": "cmpl-6i7UeOPvGYMB5TZZ7D5lWjNosbRkH",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}
------------------------------------------


This is a test
This is a sentence
This is a text


100%|██████████| 11/11 [00:04<00:00,  2.63it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
    }
  ],
  "created": 1675973257,
  "id": "cmpl-6i7UfmcOSvef5PtRZDK6vhgyB94wv",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}
------------------------------------------


This is a test
This is a sentence
This is a text


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7Ub8QqVgM5dv8L7nIsMbldi7VPY at 0x2b989f8ac830> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
     }
   ],
   "created": 1675973253,
   "id": "cmpl-6i7Ub8QqVgM5dv8L7nIsMbldi7VPY",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 16,
     "prompt_tokens": 17,
     "total_tokens": 33
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UbrZn4hgrDNukvQuRtUH7ZHbSV at 0x2b989f8ac990> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nThis is a test\nThis is a sentence\nThis is a text"
     }
   ],
   "created": 1675973253,
   "id": "cmpl-6i7UbrZn4hgrDNukvQuRtUH7ZHbSV",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage

The output there is a little weird, as it extracts things that aren't in the text it was asked to extract from, but it does something, which at this moment is all I'm really looking for. The weirder thing is that it worked correctly when I had mispelled semantic as "semanttic", as soon as I took out the extra "t" it started going wonky. Will leave as is for now, but that seems to be a reproducible behavior, so I'll keep it in mind for the future.

In [8]:
prompt_step2 = ('Extract the semantic triples from the following text: "This is a test". '
                'Example triple: ("This", "is a", "test")')
gpt3_predict(abstracts, prompt_step2, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:01,  5.05it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973257,
  "id": "cmpl-6i7UfxIMENN0F4ljlRhRB4Xf0CI58",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 30
  }
}
------------------------------------------



 18%|█▊        | 2/11 [00:00<00:01,  4.96it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973257,
  "id": "cmpl-6i7UfPUJm3FduGjrKHWCwLM702nvR",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 30
  }
}
------------------------------------------



 27%|██▋       | 3/11 [00:00<00:01,  4.44it/s]

...


Response and prediction for PMID16657524_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973259,
  "id": "cmpl-6i7UhQ2asEPNr9T5CGUn1vgUERs4V",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 30
  }
}
------------------------------------------



100%|██████████| 11/11 [00:02<00:00,  4.67it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973259,
  "id": "cmpl-6i7UhtI3GAVzgIrd2Iz1vB1Q7lgQA",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 30
  }
}
------------------------------------------


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973260,
  "id": "cmpl-6i7Ui9XcEkKyGvDQS8PnlQlHTGUMY",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 30
  }
}
------------------------------------------



100%|██████████| 11/11 [00:02<00:00,  4.31it/s]


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UfxIMENN0F4ljlRhRB4Xf0CI58 at 0x2b981c824728> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": ""
     }
   ],
   "created": 1675973257,
   "id": "cmpl-6i7UfxIMENN0F4ljlRhRB4Xf0CI58",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "prompt_tokens": 30,
     "total_tokens": 30
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7UfPUJm3FduGjrKHWCwLM702nvR at 0x2b989f8ac5c8> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": ""
     }
   ],
   "created": 1675973257,
   "id": "cmpl-6i7UfPUJm3FduGjrKHWCwLM702nvR",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "prompt_tokens": 30,
     "total_tokens": 30
   }
 },
 'PMID15869647_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7Ug7

There are no outputs as soon as I put in the example; this appears to be the point of failiure, Let's try another escalation in a different direction to test that theory:

In [11]:
prompt_step2b = ('Extract the biological relationships from the following text: '
                '"Jasmonic acid is a plant hormone. It regulates drought response."')
gpt3_predict(abstracts, prompt_step2b, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:07,  1.31it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
    }
  ],
  "created": 1675973801,
  "id": "cmpl-6i7dRylERJxPFNOlchqXQBuNn8oL2",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 26,
    "total_tokens": 42
  }
}
------------------------------------------


Jasmonic acid is a plant hormone that regulates drought response.


 18%|█▊        | 2/11 [00:01<00:05,  1.51it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
    }
  ],
  "created": 1675973802,
  "id": "cmpl-6i7dS2IwwZEH8rXoHaSLhuI09GAmJ",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 26,
    "total_tokens": 42
  }
}
------------------------------------------


Jasmonic acid is a plant hormone that regulates drought response.


 27%|██▋       | 3/11 [00:01<00:04,  1.73it/s]

...

 91%|█████████ | 10/11 [00:04<00:00,  2.29it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
    }
  ],
  "created": 1675973805,
  "id": "cmpl-6i7dV7WfO5pkujoz0OCOIztVJgYCR",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 26,
    "total_tokens": 42
  }
}
------------------------------------------


Jasmonic acid is a plant hormone that regulates drought response.


100%|██████████| 11/11 [00:05<00:00,  2.18it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
    }
  ],
  "created": 1675973805,
  "id": "cmpl-6i7dVpTcjwAXYPmLwZrPSxjXRXs8N",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 26,
    "total_tokens": 42
  }
}
------------------------------------------


Jasmonic acid is a plant hormone that regulates drought response.


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dRylERJxPFNOlchqXQBuNn8oL2 at 0x2b989f914d00> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
     }
   ],
   "created": 1675973801,
   "id": "cmpl-6i7dRylERJxPFNOlchqXQBuNn8oL2",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 16,
     "prompt_tokens": 26,
     "total_tokens": 42
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dS2IwwZEH8rXoHaSLhuI09GAmJ at 0x2b989f914db0> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid is a plant hormone that regulates drought response."
     }
   ],
   "created": 1675973802,
   "id": "cmpl-6i7dS2IwwZEH8rXoHaSLhuI09GAmJ",
   "model": "text-curie-001",
   "object"

In [12]:
prompt_step3b = ('Extract the biological relationships from the following text as '
                '(Subject, Predicate, Object) triples: '
                '"Jasmonic acid is a plant hormone. It regulates drought response."')
gpt3_predict(abstracts, prompt_step3b, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:03,  2.92it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
    }
  ],
  "created": 1675973807,
  "id": "cmpl-6i7dXGb5BcuEiqJZSNjTD44KsxdaP",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 37,
    "total_tokens": 53
  }
}
------------------------------------------


Jasmonic acid, a plant hormone, regulates drought response.


 18%|█▊        | 2/11 [00:00<00:03,  2.89it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
    }
  ],
  "created": 1675973808,
  "id": "cmpl-6i7dYZFzLgU3evJ8OURqupgZPPmgB",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 37,
    "total_tokens": 53
  }
}
------------------------------------------


Jasmonic acid, a plant hormone, regulates drought response.


 27%|██▋       | 3/11 [00:01<00:02,  2.75it/s]

...

 91%|█████████ | 10/11 [00:03<00:00,  2.71it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
    }
  ],
  "created": 1675973811,
  "id": "cmpl-6i7dbntm7pIjsT7susSeyfATobVq3",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 37,
    "total_tokens": 53
  }
}
------------------------------------------


Jasmonic acid, a plant hormone, regulates drought response.


100%|██████████| 11/11 [00:04<00:00,  2.67it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
    }
  ],
  "created": 1675973811,
  "id": "cmpl-6i7dbZXNq4eqK1ablt0NzVZKc6MKm",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 37,
    "total_tokens": 53
  }
}
------------------------------------------


Jasmonic acid, a plant hormone, regulates drought response.


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dXGb5BcuEiqJZSNjTD44KsxdaP at 0x2b989f9cb3b8> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
     }
   ],
   "created": 1675973807,
   "id": "cmpl-6i7dXGb5BcuEiqJZSNjTD44KsxdaP",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 16,
     "prompt_tokens": 37,
     "total_tokens": 53
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dYZFzLgU3evJ8OURqupgZPPmgB at 0x2b989f9cb6d0> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid, a plant hormone, regulates drought response."
     }
   ],
   "created": 1675973808,
   "id": "cmpl-6i7dYZFzLgU3evJ8OURqupgZPPmgB",
   "model": "text-curie-001",
   "object": "text_comp

In [13]:
prompt_step4b = ('Extract the biological relationships from the following text as '
                '("Subject", "Predicate", "Object") triples: '
                '"Jasmonic acid is a plant hormone. It regulates drought response."')
gpt3_predict(abstracts, prompt_step4b, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:04,  2.11it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
    }
  ],
  "created": 1675973812,
  "id": "cmpl-6i7dcJZT5ZgPtc7OxHNBq1BGqtbpp",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 39,
    "total_tokens": 66
  }
}
------------------------------------------


Subject: Jasmonic acid
Predicate: is a plant hormone
Object: Jasmonic acid regulates drought response.


 18%|█▊        | 2/11 [00:00<00:04,  2.21it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
    }
  ],
  "created": 1675973812,
  "id": "cmpl-6i7dcYSRgpLBp2LPFAuCbFGKRcQTc",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 39,
    "total_tokens": 66
  }
}
------------------------------------------


Subject: Jasmonic acid
Predicate: is a plant hormone
Object: Jasmonic acid regulates drought response.


 27%|██▋       | 3/11 [00:01<00:03,  2.23it/s]

...

 91%|█████████ | 10/11 [00:04<00:00,  2.45it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
    }
  ],
  "created": 1675973815,
  "id": "cmpl-6i7dfUVIhHP5s2AESRCXu1HlsJT4D",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 39,
    "total_tokens": 66
  }
}
------------------------------------------


Subject: Jasmonic acid
Predicate: is a plant hormone
Object: Jasmonic acid regulates drought response.


100%|██████████| 11/11 [00:04<00:00,  2.24it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
    }
  ],
  "created": 1675973816,
  "id": "cmpl-6i7dgYrfgv0TMmHfo5VvsyYlgGHmh",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 39,
    "total_tokens": 66
  }
}
------------------------------------------


Subject: Jasmonic acid
Predicate: is a plant hormone
Object: Jasmonic acid regulates drought response.


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dcJZT5ZgPtc7OxHNBq1BGqtbpp at 0x2b989f8efd00> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
     }
   ],
   "created": 1675973812,
   "id": "cmpl-6i7dcJZT5ZgPtc7OxHNBq1BGqtbpp",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 27,
     "prompt_tokens": 39,
     "total_tokens": 66
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dcYSRgpLBp2LPFAuCbFGKRcQTc at 0x2b989f9cbdb0> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nSubject: Jasmonic acid\nPredicate: is a plant hormone\nObject: Jasmonic acid regulates drought response."
     }
   ],
   "created": 1675973812,
   "id": "

Adding the quotes within the beginning of the prompt does get it to tell me which parts are SP and O, but it's not quite the format that I'm looking for. What id I ask it to format the response a certain way?

In [14]:
prompt_step5b = ('Extract the biological relationships from the following text as '
                '(Subject, Predicate, Object) triples: '
                '"Jasmonic acid is a plant hormone. It regulates drought response." '
                'Please format the response using the following format: '
                '("Subject", "Predicate", "Object")')
gpt3_predict(abstracts, prompt_step5b, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:03,  2.60it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
    }
  ],
  "created": 1675973817,
  "id": "cmpl-6i7dhNkZ7bnqFUgI20jlemgRVXyEb",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 56,
    "total_tokens": 73
  }
}
------------------------------------------


Jasmonic acid is a plant hormone. It regulates drought response.


 18%|█▊        | 2/11 [00:00<00:03,  2.68it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
    }
  ],
  "created": 1675973817,
  "id": "cmpl-6i7dhDYVaex149Ktr9RmNAClPJVHP",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 56,
    "total_tokens": 73
  }
}
------------------------------------------


Jasmonic acid is a plant hormone. It regulates drought response.


 27%|██▋       | 3/11 [00:01<00:03,  2.63it/s]

...

 91%|█████████ | 10/11 [00:04<00:00,  2.31it/s]


Response and prediction for PMID28584068_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
    }
  ],
  "created": 1675973820,
  "id": "cmpl-6i7dks0fZJRZlCajMRIumTt9cNkHe",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 56,
    "total_tokens": 73
  }
}
------------------------------------------


Jasmonic acid is a plant hormone. It regulates drought response.


100%|██████████| 11/11 [00:04<00:00,  2.47it/s]


Response and prediction for PMID18941053_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
    }
  ],
  "created": 1675973821,
  "id": "cmpl-6i7dlC6PrTfOCqYpIwjaYuG04qgH9",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 56,
    "total_tokens": 73
  }
}
------------------------------------------


Jasmonic acid is a plant hormone. It regulates drought response.


{'PMID25914698_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dhNkZ7bnqFUgI20jlemgRVXyEb at 0x2b989f9f8410> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
     }
   ],
   "created": 1675973817,
   "id": "cmpl-6i7dhNkZ7bnqFUgI20jlemgRVXyEb",
   "model": "text-curie-001",
   "object": "text_completion",
   "usage": {
     "completion_tokens": 17,
     "prompt_tokens": 56,
     "total_tokens": 73
   }
 },
 'PMID16663540_abstract.txt': <OpenAIObject text_completion id=cmpl-6i7dhDYVaex149Ktr9RmNAClPJVHP at 0x2b989f9f8728> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "logprobs": null,
       "text": "\n\nJasmonic acid is a plant hormone. It regulates drought response."
     }
   ],
   "created": 1675973817,
   "id": "cmpl-6i7dhDYVaex149Ktr9RmNAClPJVHP",
   "model": "text-curie-001",
   "object": 

Somehow that makes it worse. For thouroughness, let's try the example again with the biological context:

In [15]:
prompt_step6b = ('Extract the biological relationships from the following text as '
                '(Subject, Predicate, Object) triples: '
                '"Jassmonic acid is a plant hormone. It regulates drought response." '
                'Example (Subject, Predicate, Object) triple: '
                '("Protein 1", "activates", "Protein 2")')
gpt3_predict(abstracts, prompt_step6b, model='text-curie-001', max_tokens=1500)

 18%|█▊        | 2/11 [00:00<00:01,  5.27it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973821,
  "id": "cmpl-6i7dlA9CyudJHWbOXycMpNmwsLed8",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973821,
  "id": "cmpl-6i7dlaYs7bbAuMBOvXV0uGqnFtJUm",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------



 27%|██▋       | 3/11 [00:00<00:01,  4.82it/s]


Response and prediction for PMID15869647_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973821,
  "id": "cmpl-6i7dlZv3xtJOWWW8cZtsDzcYbsp5U",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------



 36%|███▋      | 4/11 [00:00<00:01,  4.68it/s]

...


Response and prediction for PMID21487047_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973822,
  "id": "cmpl-6i7dmdtTj6L2TmC30TnwK2pdrpaPX",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------



 64%|██████▎   | 7/11 [00:01<00:00,  4.82it/s]


Response and prediction for PMID30677553_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973822,
  "id": "cmpl-6i7dmdDOKBnR0no8WiFlco0jSMVGk",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------


Response and prediction for PMID30272908_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1675973822,
  "id": "cmpl-6i7dm7ac8EWSjHgW7WA3nWNbYCSpw",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "prompt_tokens": 62,
    "total_tokens": 62
  }
}
------------------------------------------



 64%|██████▎   | 7/11 [00:01<00:00,  4.54it/s]


RateLimitError: Rate limit reached for default-global-with-image-limits in organization org-PIl5EAsmb9MK5zFMsAtzyUqU on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

Nothing. I am utterly confused about why this is happening, as it worked really well two days ago. However, two days ago I also tried [this prompt format](https://community.openai.com/t/fine-tuning-gpt-3-for-consistent-output-format/46716) and it did nothing, maybe it's worth trying that again:

In [16]:
prompt_step2c = ('Extract the (subject, predicate, object) triples from the following text: '
                 '"Jasmonic acid is a plant hormone. It regulates drought response."'
                 '. Please use the format template for each triple.\n'
                 '---BEGIN FORMAT TEMPLATE---\n'
                 '(“${SUBJECT}”, “${PREDICATE}”, “${OBJECT}”)\n'
                 '---END FORMAT TEMPLATE---\n'
                 '---BEGIN EXTRACTING TRIPLES---')
gpt3_predict(abstracts, prompt_step2c, model='text-curie-001', max_tokens=1500)

  9%|▉         | 1/11 [00:00<00:03,  2.73it/s]


Response and prediction for PMID25914698_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nJasmonic acid is a plant hormone.\nIt regulates drought response."
    }
  ],
  "created": 1675973830,
  "id": "cmpl-6i7duFI70NxroSAHdaVk5z6yTRj3W",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 108,
    "total_tokens": 125
  }
}
------------------------------------------

Jasmonic acid is a plant hormone.
It regulates drought response.


 18%|█▊        | 2/11 [00:00<00:03,  2.76it/s]


Response and prediction for PMID16663540_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nJasmonic acid is a plant hormone.\nIt regulates drought response."
    }
  ],
  "created": 1675973831,
  "id": "cmpl-6i7dvzde7RFHwPqAQ1o1EiVbFgrSL",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 108,
    "total_tokens": 125
  }
}
------------------------------------------

Jasmonic acid is a plant hormone.
It regulates drought response.


 27%|██▋       | 3/11 [00:01<00:02,  2.74it/s]


Response and prediction for PMID15869647_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nJasmonic acid is a plant hormone.\nIt regulates drought response."
    }
  ],
  "created": 1675973831,
  "id": "cmpl-6i7dvxNUjYjeOiodstsbEcn2Dojrw",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 108,
    "total_tokens": 125
  }
}
------------------------------------------

Jasmonic acid is a plant hormone.
It regulates drought response.


 36%|███▋      | 4/11 [00:01<00:02,  2.75it/s]


Response and prediction for PMID1161793_abstract.txt:
------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nJasmonic acid is a plant hormone.\nIt regulates drought response."
    }
  ],
  "created": 1675973832,
  "id": "cmpl-6i7dw4dbwuSodyKprPq4zuBx6kLUN",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 108,
    "total_tokens": 125
  }
}
------------------------------------------

Jasmonic acid is a plant hormone.
It regulates drought response.


RateLimitError: Rate limit reached for default-global-with-image-limits in organization org-PIl5EAsmb9MK5zFMsAtzyUqU on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

That template didn't do anything helpful, but I still got an output. What is it about the example I provide that makes it output nothing?